# Credit Card Fraud Detection - Complete EDA
## Task 1: EDA and Preprocessing for CreditCard Dataset

This notebook analyzes the creditcard.csv dataset with:
- Comprehensive EDA
- Missing value analysis
- Feature scaling
- Class imbalance handling

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('Libraries imported successfully!')

## 1. Data Loading

In [ ]:
# Load data
DATA_PATH = '../data/raw/creditcard.csv'
df = pd.read_csv(DATA_PATH)

print(f'✓ Data loaded successfully!')
print(f'Shape: {df.shape}')
print(f'\nFirst few rows:')
df.head()

## 2. Data Cleaning
### 2.1 Missing Values

In [ ]:
# Check for missing values
print('Missing Values Analysis:')
print('='*50)
missing = df.isnull().sum()
print(missing[missing > 0])

if missing.sum() == 0:
    print('\n✓ No missing values found!')
else:
 print(f'\n⚠ Total missing: {missing.sum()}')

# Data types
print('\nData Types:')
print(df.dtypes.value_counts())

# Basic statistics
print('\nBasic Info:')
print(df.info())

### 2.2 Remove Duplicates

In [ ]:
# Check and remove duplicates
initial_rows = len(df)
df.drop_duplicates(inplace=True)
duplicates_removed = initial_rows - len(df)

print(f'Duplicates removed: {duplicates_removed}')
print(f'Final shape: {df.shape}')

## 3. Exploratory Data Analysis
### 3.1 Class Distribution

In [ ]:
# Class distribution
class_dist = df['Class'].value_counts()
class_pct = df['Class'].value_counts(normalize=True) * 100

print('Class Distribution:')
print('='*50)
print(f'Normal (0): {class_dist[0]:,} ({class_pct[0]:.4f}%)')
print(f'Fraud (1): {class_dist[1]:,} ({class_pct[1]:.4f}%)')
print(f'\n⚠ Imbalance Ratio: {class_dist[0]/class_dist[1]:.0f}:1')
print('This is a HIGHLY imbalanced dataset!')

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
sns.countplot(data=df, x='Class', ax=ax1)
ax1.set_title('Class Distribution', fontsize=14, fontweight='bold')
ax1.set_xticklabels(['Normal', 'Fraud'])
for p in ax1.patches:
    ax1.annotate(f'{int(p.get_height()):,}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom')

# Pie chart (log scale for visibility)
ax2.pie(class_dist, labels=['Normal', 'Fraud'], autopct='%1.4f%%', startangle=90)
ax2.set_title('Class Percentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 3.2 Time and Amount Analysis

In [ ]:
# Time and Amount statistics
print('Time and Amount Statistics:')
print('='*50)
print('\nTime (seconds from first transaction):')
print(df['Time'].describe())
print('\nAmount:')
print(df['Amount'].describe())

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time distribution
axes[0, 0].hist(df['Time'], bins=50, edgecolor='black')
axes[0, 0].set_title('Time Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Time (seconds)')
axes[0, 0].set_ylabel('Frequency')

# Amount distribution (log scale)
axes[0, 1].hist(df[df['Amount'] > 0]['Amount'], bins=50, edgecolor='black')
axes[0, 1].set_yscale('log')
axes[0, 1].set_title('Amount Distribution (log scale)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Amount')

# Amount by class
df.boxplot(column='Amount', by='Class', ax=axes[1, 0])
axes[1, 0].set_title('Amount by Class', fontsize=12, fontweight='bold')
axes[1, 0].set_xticklabels(['Normal', 'Fraud'])

# Time by class
df.boxplot(column='Time', by='Class', ax=axes[1, 1])
axes[1, 1].set_title('Time by Class', fontsize=12, fontweight='bold')
axes[1, 1].set_xticklabels(['Normal', 'Fraud'])

plt.tight_layout()
plt.show()

### 3.3 PCA Feature Analysis

Note: V1-V28 are PCA-transformed features for confidentiality

In [ ]:
# Analyze PCA features
v_features = [f'V{i}' for i in range(1, 29)]

# Show a subset of V features distributions
fig, axes = plt.subplots(3, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, feature in enumerate(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']):
    # Plot distribution by class
    df[df['Class'] == 0][feature].hist(bins=50, alpha=0.5, label='Normal', ax=axes[idx])
    df[df['Class'] == 1][feature].hist(bins=50, alpha=0.5, label='Fraud', ax=axes[idx])
    axes[idx].set_title(f'{feature} Distribution by Class', fontsize=10)
    axes[idx].legend()

plt.tight_layout()
plt.show()

print('\nObservation: Some PCA features show clear separation between fraud and normal transactions')

### 3.4 Correlation Analysis

In [ ]:
# Correlation with target
correlations = df.corr()['Class'].sort_values(ascending=False)
print('Top 10 features correlated with fraud:')
print(correlations.head(11))  # 11 to include Class itself

print('\nBottom 10 features (negative correlation):')
print(correlations.tail(10))

# Visualize top correlations
fig, ax = plt.subplots(figsize=(10, 8))
top_features = correlations.abs().sort_values(ascending=False)[1:16]  # Top 15 excluding Class
top_features.plot(kind='barh', ax=ax)
ax.set_title('Top 15 Features by Absolute Correlation with Fraud', fontsize=14, fontweight='bold')
ax.set_xlabel('Absolute Correlation')
plt.tight_layout()
plt.show()

## 4. Feature Scaling

In [ ]:
# Scale Time and Amount using RobustScaler (less sensitive to outliers)
print('Scaling Amount and Time features...')

rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1, 1))

# Drop original features
df.drop(['Time', 'Amount'], axis=1, inplace=True)

# Reorder columns
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']
df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

print('✓ Features scaled successfully!')
print(f'New shape: {df.shape}')
print('\nFirst few rows after scaling:')
df.head()

## 5. Final Dataset Overview

In [ ]:
# Final summary
print('FINAL DATASET SUMMARY')
print('='*50)
print(f'Total transactions: {len(df):,}')
print(f'Total features: {df.shape[1]}')
print(f'\nClass Distribution:')
print(df['Class'].value_counts())
print(f'\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')

print('\n✓ Dataset ready for modeling!')
print('\nNote: This dataset is PCA-transformed and highly imbalanced.')
print('SMOTE or other resampling techniques recommended before training.')

## 6. Save Processed Data

In [ ]:
# Save processed data
import os
PROCESSED_DIR = '../data/processed'
os.makedirs(PROCESSED_DIR, exist_ok=True)

df.to_csv(f'{PROCESSED_DIR}/creditcard_processed.csv', index=False)
print(f'✓ Processed data saved to: {PROCESSED_DIR}/creditcard_processed.csv')

## Summary

### CreditCard Dataset Analysis Complete:

✅ **Data Cleaning**
- No missing values
- Duplicates removed
- Data types verified

✅ **EDA Completed**
- Extreme class imbalance identified (99.83% vs 0.17%)
- PCA features analyzed
- Correlation with fraud identified
-Time and Amount patterns explored

✅ **Feature Engineering**
- RobustScaler applied to Time and Amount
- Features ready for modeling

### Key Findings:
1. **Severe Class Imbalance**: Only 0.17% fraud - requires SMOTE/resampling
2. **PCA Features**: Already dimensionality-reduced for privacy
3. **Important Features**: V17, V14, V12, V10 show strong correlation with fraud
4. **Amount**: Fraud transactions tend to have different amount patterns

### Next Steps:
- Apply SMOTE before training
- Train classification models
- Focus on Recall and F1-score metrics